# Sybot bug reproducer

Enter the URL of a syzbot bug you'd like to reproduce:

In [ ]:
bug_url=input('Syzbot bug URL, e.g.: https://syzkaller.appspot.com/bug?extid=fcc47ba2476570cbbeb0')

## Crash selection

Download the syzbot page as HTML.

In [ ]:
bug_html=!curl {bug_url}

Parse the crashes table as a Pandas data frame and pretty print interesting fields

In [ ]:
import pandas as pd
crashes = pd.read_html(bug_html.nlstr, match="Crashes", extract_links="body")[0]
display(crashes.applymap(lambda val: val[0])[["Time", "Commit", "Syz repro", "C repro"]])

Select the crash you'd like to reproduce:

In [ ]:
crash_id=int(input("Crash to reproduce (by row number), e.g.: 0"))

## Kernel setup

Checkout the kernel commit

In [ ]:
!git checkout {crashes["Commit"][crash_id][0]}

Overwrite the local `.config`

In [ ]:
!curl "{'https://syzkaller.appspot.com' + crashes['Config'][crash_id][1]}" > ../.config

Build

In [ ]:
!SPINNER=0 ./tasks.sh build

Start the kernel by pressing `F5`

## C reproducer

Download the code

In [ ]:
!curl "{'https://syzkaller.appspot.com' + crashes['C repro'][crash_id][1]}" > /tmp/syzbot-repro.c

Build it

In [ ]:
!clang -static -lpthread /tmp/syzbot-repro.c -o /tmp/syzbot-repro

Send it to the VM

In [ ]:
!./tasks.sh push /tmp/syzbot-repro

Run it

In [ ]:
!./tasks.sh run /root/syzbot-repro

## Syz reproducer

Download the code

In [ ]:
!curl "{'https://syzkaller.appspot.com' + crashes['Syz repro'][crash_id][1]}" > /tmp/syzbot-repro.syz

Send it to the VM

In [ ]:
!./tasks.sh push /tmp/syzbot-repro.syz

Run it (assumes the VM already has a `/usr/bin/syz-execprog`)

In [ ]:
!./tasks.sh run "syz-execprog -repeat=0 -procs=1 -sandbox='' /root/syzbot-repro.syz"